In [1]:
# Import Library for Writing to Excel Spreadsheet.
from openpyxl import load_workbook
# Import Library for Accessing XML File.
import xml.etree.ElementTree as ET
# Deal with Tabular Data using Pandas Library.
import pandas as pd

In [2]:
raw_dataframe = pd.read_excel('test_excel_1.xlsx')

In [3]:
# This function shortens an id from the Excel file.
def shorten_id(original_column) :
    length_to_cut = len('Delete Account ')
    length_total = len(original_column)
    truncated_column = original_column[length_to_cut: length_total]

    return truncated_column

In [4]:
raw_dataframe['Col1'] = raw_dataframe['Col1'].apply(shorten_id)

In [5]:
id_series = raw_dataframe['Col1']
submission_date_series = raw_dataframe['Col5']
due_date_series = raw_dataframe['Col6'] 

In [6]:
tree = ET.parse('test_xml.xml')

In [7]:
root = tree.getroot()

In [8]:
role_column = []
organized_role_column = []
xml_ids = []

In [9]:
def shorten_role(original_column) :
    length_to_cut = len('Role=')
    length_total = len(original_column)
    truncated_column = original_column[length_to_cut: length_total]

    return truncated_column

In [10]:
# Iterate through XML file to filter and organize relevant information.
for account in root.iter('account') :
    id = account.get('id', default = None)
    
    for i in range(id_series.size) :
        # If ID is found in the ID series, execute code.
        if (id == id_series[i]) :
            # Add this unordered ID to the list.
            xml_ids.append(id)
            ## The following lines are commented out -> used for debugging loop.
            ## print(id)
            ## print(id_series[i])
            ## print('\n')

            # Assign role column to have the role information.
            for attribute in account.iter('attribute') :
                ## The following lines are commented out -> used for debugging loop.
                ## print(attribute.attrib)
                ## print('\n')

                if attribute.get('name') == 'Role' :
                    attributeValueRef_id = str()

                    for attributeValueRef in attribute.iter('attributeValueRef') :
                        ## The following line is commented out -> used for debugging loop.
                        ## print(attributeValueRef.attrib)
                        
                        # Remove the 'Role=' using the custom function before acquiring the information.
                        attributeValueRef_id += shorten_role(attributeValueRef.get('id')) + ' \n'
                    # Append each string to the list if the ID is found in the ID series.
                    role_column.append(attributeValueRef_id)

                    ## The following lines are commented out -> used for debugging loop.
                    ## print('\n')
                    ## print(role_column)
                    ## print('\n')

# Organize role column to match the ordered ID series.
for organized_index in range(id_series.size) :
    ## The following lines are commented out -> used for debugging loop. 
    ## print(organized_index)
    ## print(id_series[organized_index])
    ## print('\n')

    # Add an empty string to end of list to account for IDs with blank roles.
    organized_role_column.append('')
    for unorganized_index in range(len(xml_ids)) :
        ## The following lines are commented out -> used for debugging loop. 
        ## print(unorganized_index)
        ## print(xml_ids[unorganized_index])
        ## print('\n')   
        
        if (id_series[organized_index] == xml_ids[unorganized_index]) :
            # Replace empty string for IDs with roles in the XML file.
            organized_role_column.insert(organized_index, (role_column[unorganized_index]))
    

In [11]:
print(organized_role_column)

['ABCDEFG \nHIJKLMN \n', 'LMNOPQRST \n', 'DSDSDSDSD \n', '******** \n', 'AQSDEWBBB \n', '', '', '', '', '']


In [12]:
organized_role_series = pd.DataFrame(organized_role_column)

In [13]:
combined_dataframe = pd.concat([id_series, organized_role_series, submission_date_series, due_date_series], axis = 1)

In [14]:
combined_dataframe.rename(columns = {'Col1' : 'ID', 0 : 'Role Information', 'Col5' : 'Submission Date', 'Col6' : 'Due Date'}, inplace = True)

In [15]:
print(combined_dataframe)

       ID      Role Information     Submission Date            Due Date
0  ASDVF5  ABCDEFG \nHIJKLMN \n 2020-08-06 08:28:00 2020-08-11 08:28:00
1  ABCDE5          LMNOPQRST \n 2020-08-06 09:30:00 2020-08-11 09:30:00
2  MNOPQ4          DSDSDSDSD \n 2020-08-06 09:39:00 2020-08-11 09:39:00
3  XTGSD2           ******** \n 2020-08-06 09:50:00 2020-08-11 09:50:00
4  XYZRF2          AQSDEWBBB \n 2020-08-06 09:50:00 2020-08-11 09:50:00
5     NaN                                       NaT                 NaT
6     NaN                                       NaT                 NaT
7     NaN                                       NaT                 NaT
8     NaN                                       NaT                 NaT
9     NaN                                       NaT                 NaT


In [16]:
writer = pd.ExcelWriter('test_excel_2.xlsx', engine = 'xlsxwriter')

In [17]:
combined_dataframe.to_excel(writer,
             sheet_name ='organized_sheet') 

In [18]:
writer.save()

In [19]:
writer.close()

C:\Users\Muntakim\AppData\Local\Programs\Python\Python38-32\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
